Expected format:
{
  "audio": "/content/keyboard_dataset/audio/example_0001.wav",
  "events": [
    {"time": 0.12, "key": "A", "type": "down"},
    {"time": 0.25, "key": "A", "type": "up"},
    {"time": 0.40, "key": "SPACE", "type": "down"},
    {"time": 0.50, "key": "SPACE", "type": "up"}
  ]
}

In [9]:
# Cell 1: Basic setup

import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)
# assert device == "cuda", "Please enable GPU in Runtime > Change runtime type."


Device: cpu


In [10]:
# Cell 1: Point to your directory and inspect JSON/audio files

import os, glob

DATA_DIR = "../recordings"  # 🔁 change this to your folder

json_files = sorted(glob.glob(os.path.join(DATA_DIR, "*.json")))
webm_files = sorted(glob.glob(os.path.join(DATA_DIR, "*.webm")))

print(f"Found {len(json_files)} JSON files")
print(f"Found {len(webm_files)} WEBM files")

print("First 3 JSON files:")
for f in json_files[:3]:
    print("  ", os.path.basename(f))

print("First 3 WEBM files:")
for f in webm_files[:3]:
    print("  ", os.path.basename(f))


Found 7 JSON files
Found 7 WEBM files
First 3 JSON files:
   recording_20251107_225557_DELETED.json
   recording_20251107_231415_DELETED.json
   recording_20251107_231933.json
First 3 WEBM files:
   recording_20251107_225557_DELETED.webm
   recording_20251107_231415_DELETED.webm
   recording_20251107_231933.webm


In [11]:
# Cell 2: Load JSON files into a Dataset and attach audio paths

from datasets import load_dataset, Audio, DatasetDict
import os

TARGET_SAMPLING_RATE = 16000  # safe to redefine, or remove if already defined

# Use a non-reserved split name like "data" instead of "all"
data_files = {"data": os.path.join(DATA_DIR, "*.json")}
raw_all = load_dataset("json", data_files=data_files)["data"]

print(raw_all)
print("Example raw JSON entry:\n", raw_all[0])

# Add an "audio" field pointing to the matching .webm file for each recording
def add_audio_path(example):
    # Use the audio_file field instead of constructing from recording_id
    audio_filename = example["audio_file"]
    audio_path = os.path.join(DATA_DIR, audio_filename)
    
    # If the file doesn't exist, check for a _DELETED version
    if not os.path.exists(audio_path):
        # Try inserting _DELETED before the extension
        base_name = audio_filename.replace(".webm", "")
        deleted_path = os.path.join(DATA_DIR, base_name + "_DELETED.webm")
        if os.path.exists(deleted_path):
            audio_path = deleted_path
        else:
            raise FileNotFoundError(f"Missing audio file: tried {audio_path} and {deleted_path}")
    
    example["audio"] = audio_path
    return example

raw_all = raw_all.map(add_audio_path)

# Cast "audio" column to an Audio feature so it gets decoded & resampled on access
raw_all = raw_all.cast_column("audio", Audio(sampling_rate=TARGET_SAMPLING_RATE))

print("\nAfter adding audio and casting column:")
print(raw_all)


Dataset({
    features: ['recording_id', 'start_timestamp', 'keyboard_session_id', 'control_session_id', 'keystrokes', 'audio_file', 'end_timestamp'],
    num_rows: 7
})
Example raw JSON entry:
 {'recording_id': '19a6c54a-8511-4e6d-a915-b6525da782b3', 'start_timestamp': 1762574157767, 'keyboard_session_id': 'ee8495b2-320b-4bfc-a5f2-180050de5064', 'control_session_id': 'ad2cc912-9a80-46ac-b597-c818175353d9', 'keystrokes': [{'timestamp': 1762574158934, 'key': ',', 'event_type': 'keydown'}, {'timestamp': 1762574158953, 'key': ',', 'event_type': 'keyup'}, {'timestamp': 1762574159030, 'key': 'j', 'event_type': 'keydown'}, {'timestamp': 1762574159046, 'key': 'h', 'event_type': 'keydown'}, {'timestamp': 1762574159051, 'key': 'j', 'event_type': 'keyup'}, {'timestamp': 1762574159073, 'key': 'h', 'event_type': 'keyup'}, {'timestamp': 1762574159102, 'key': 'k', 'event_type': 'keydown'}, {'timestamp': 1762574159108, 'key': 'j', 'event_type': 'keydown'}, {'timestamp': 1762574159151, 'key': 'j', 'ev

In [12]:
# Cell 3: Create train/validation splits and inspect a couple of examples

# 90/10 split into train / validation
splits = raw_all.train_test_split(test_size=0.1, seed=42)
raw_datasets = DatasetDict({
    "train": splits["train"],
    "validation": splits["test"],
})

print(raw_datasets)

# Peek at one training example
ex = raw_datasets["train"][0]
print("\nOne training example:")
print("recording_id:", ex["recording_id"])
print("audio object keys:", ex["audio"].keys())
print("audio sampling rate:", ex["audio"]["sampling_rate"])
print("audio array shape:", ex["audio"]["array"].shape)
print("num keystrokes:", len(ex["keystrokes"]))
print("first 5 keystrokes:", ex["keystrokes"][:5])


DatasetDict({
    train: Dataset({
        features: ['recording_id', 'start_timestamp', 'keyboard_session_id', 'control_session_id', 'keystrokes', 'audio_file', 'end_timestamp', 'audio'],
        num_rows: 6
    })
    validation: Dataset({
        features: ['recording_id', 'start_timestamp', 'keyboard_session_id', 'control_session_id', 'keystrokes', 'audio_file', 'end_timestamp', 'audio'],
        num_rows: 1
    })
})


RuntimeError: Could not load libtorchcodec. Likely causes:
          1. FFmpeg is not properly installed in your environment. We support
             versions 4, 5, 6, and 7 on all platforms, and 8 on Mac and Linux.
          2. The PyTorch version (2.9.0+cu128) is not compatible with
             this version of TorchCodec. Refer to the version compatibility
             table:
             https://github.com/pytorch/torchcodec?tab=readme-ov-file#installing-torchcodec.
          3. Another runtime dependency; see exceptions below.
        The following exceptions were raised as we tried to load libtorchcodec:
        
[start of libtorchcodec loading traceback]
FFmpeg version 8: Could not load this library: /home/zyansheep/Projects/kasca-esque/.venv/lib/python3.13/site-packages/torchcodec/libtorchcodec_core8.so
FFmpeg version 7: Could not load this library: /home/zyansheep/Projects/kasca-esque/.venv/lib/python3.13/site-packages/torchcodec/libtorchcodec_core7.so
FFmpeg version 6: Could 